In [14]:
f = open(r"C:\Users\Shabaan\Downloads\ghalib.txt", "r", encoding="utf-8")
token = f.read()
f.close()

In [15]:
tokens=list(token)
len(tokens)

68846

In [16]:
pairs=list(zip(tokens,tokens[1:]))
print(pairs[0:5])

[('ب', 'ا'), ('ا', 'ز'), ('ز', 'ی'), ('ی', 'چ'), ('چ', 'ۂ')]


In [17]:
def getpfreq(tokens):
    counts = {}
    for p in zip(tokens,tokens[1:]):
        counts[p] = counts.get(p, 0) + 1  
    return counts
pf=getpfreq(pairs)
print(list(pf.items())[:5])

[((('ب', 'ا'), ('ا', 'ز')), 21), ((('ا', 'ز'), ('ز', 'ی')), 12), ((('ز', 'ی'), ('ی', 'چ')), 1), ((('ی', 'چ'), ('چ', 'ۂ')), 1), ((('چ', 'ۂ'), ('ۂ', ' ')), 5)]


In [18]:
spf=sorted(((v, k) for k, v in pf.items()), reverse=True)
spf[0]

(659, (('ی', 'ں'), ('ں', ' ')))

In [19]:
def merge(cipher,pair,nt):
    newcipher=[]
    i=0
    while i < len(cipher):
        if i< len(cipher)-1 and cipher[i]==pair[0] and cipher[i+1]== pair[1]:
            newcipher.append(nt)
            i+=2
        else:
            newcipher.append(cipher[i])
            i+=1
    return newcipher

In [20]:
merge([4,5,6,5,6,5,6,4,7,8,6,5,7],[6,5],10)

[4, 5, 10, 10, 6, 4, 7, 8, 10, 7]

In [30]:
vocab_size = 300
num_merges = vocab_size - 256
cipher = list(token.encode('utf-8'))

In [32]:
def BPE(corpus,num_merges):
    
    merges={}
    vocab = {t:bytes([t]) for t in range(256)}
    newcipher = corpus
    for i in range(num_merges):
        
        pf = getpfreq(newcipher)
        mp = max(pf, key=pf.get)
        nt = 256 + i
        
        print(f"merging {mp} into a new token {nt}")
        newcipher = merge(newcipher,mp,nt)
        merges[mp] = nt  

        vocab[nt] = vocab[mp[0]] + vocab[mp[1]]

        
    return vocab,merges

vocab , merges= BPE(cipher,num_merges)
print(list(merges.items())[:5])

merging (32, 216) into a new token256
merging (216, 167) into a new token257
merging (219, 140) into a new token258
merging (219, 129) into a new token259
merging (32, 217) into a new token260
merging (216, 177) into a new token261
merging (217, 136) into a new token262
merging (218, 169) into a new token263
merging (219, 146) into a new token264
merging (32, 263) into a new token265
merging (218, 186) into a new token266
merging (217, 134) into a new token267
merging (32, 259) into a new token268
merging (217, 133) into a new token269
merging (216, 170) into a new token270
merging (217, 132) into a new token271
merging (10, 10) into a new token272
merging (216, 175) into a new token273
merging (218, 190) into a new token274
merging (260, 133) into a new token275
merging (258, 266) into a new token276
merging (218, 175) into a new token277
merging (217, 144) into a new token278
merging (260, 134) into a new token279
merging (216, 168) into a new token280
merging (216, 179) into a new t

In [41]:
def encode(text):
    cipher = list(text.encode('utf-8'))
    while len(cipher)>=2:
        pf = getpfreq(cipher)
        mp = min(pf, key=lambda p : merges.get(p,float('inf')))
        if mp not in merges:
            break
        nt = merges[mp]
        cipher = merge(cipher,mp,nt)
    return cipher

In [42]:
print(encode('کیا حال ہے'))

[263, 289, 256, 173, 257, 271, 268, 264]


In [47]:
def decode(cipher):
    text=b"".join(vocab[c] for c in cipher)
    text = text.decode('utf-8', errors ='replace')
    return text
decode([263, 289, 256, 173, 257, 271, 128, 264])

'کیا حال�ے'